## Supplementary Material
Deep Learning in EEG-Based BCIs: A Comprehensive Review of Transformer Models, Advantages, Challenges, and Applications


### EEGTransformer Class

The `EEGTransformer` class is designed to leverage a transformer-based architecture tailored specifically for Electroencephalogram (EEG) data processing.

#### Parameters:
- `num_channels` (int): Specifies the number of channels in the EEG dataset.
- `num_timepoints` (int): Indicates the number of time points or the sequence length in the EEG data.
- `output_dim` (int): Defines the output dimensionality for the classifier layer.
- `hidden_dim` (int): Specifies the hidden layer dimensionality.
- `num_heads` (int): Determines the number of attention heads to be used in the multi-head self-attention mechanism.
- `key_query_dim` (int): Denotes the dimensionality for the key/query pairs in the self-attention mechanism.
- `hidden_ffn_dim` (int): Indicates the hidden layer dimensionality for the feed-forward network.
- `intermediate_dim` (int): Refers to the dimensionality of the intermediate layer in the feed-forward network.
- `ffn_output_dim` (int): Specifies the output size of the feed-forward network.

#### Attributes:
- `positional_encoding` (torch.Tensor): A tensor of shape `(num_channels, num_timepoints)` that imparts the sequence position information.
- `multihead_attn` (nn.MultiheadAttention): Implements the multi-head self-attention mechanism.
- `ffn` (nn.Sequential): Constructs a feed-forward network composed of a linear transformation followed by ReLU activation and another linear transformation.
- `norm1` and `norm2` (nn.LayerNorm): Execute layer normalization.
- `classifier` (nn.Linear): Deploys a final linear transformation layer to categorize the input into designated classes.

#### Methods:
- `forward(X)`: Outlines the forward propagation for the model.
  - `X` (torch.Tensor): The input tensor for EEG data, which should have a shape of `(batch_size, num_channels, num_timepoints)`.

  - **Steps**:
    1. Standardize the input tensor.
    2. Apply positional encoding.
    3. Implement multi-head self-attention.
    4. Reshape the attention output and apply layer normalization.
    5. Forward the data through the feed-forward network.
    6. Flatten the resultant tensor and direct it through a classifier layer.
    7. Yield the final output.
  
### Notes:

- The model applies layer normalization after the multi-head self-attention and feed-forward network stages.
- Positional encoding is utilized to impart sequence position information to the model, which can either be relative or absolute.
- The classifier layer flattens the model output and categorizes it into `output_dim` classes.

### Usage:

To employ the `EEGTransformer` model, instantiate the class using the desired parameters. Then, similar to any other PyTorch model, forward the input data to the model and utilize the returned output for either training or inference.

```python
# Sample Usage
model = EEGTransformer(num_channels=32, num_timepoints=200, output_dim=2,
                       hidden_dim=512, num_heads=8, key_query_dim=512,
                       hidden_ffn_dim=512, intermediate_dim=2048,
                       ffn_output_dim=32)
                       
input_data = torch.randn(64, 32, 200)
output = model(input_data)
```

Ensure that the model is paired with a compatible loss function and optimizer for effective training. Depending on the specifics of the EEG dataset or application requirements, the model can be further refined.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

class EEGTransformer(nn.Module):
    def __init__(self, num_channels, num_timepoints, output_dim,
                 hidden_dim, num_heads, key_query_dim,
                 hidden_ffn_dim, intermediate_dim, ffn_output_dim):
        super(EEGTransformer, self).__init__()

        # Positional Encoding
        self.positional_encoding = torch.zeros(num_channels, num_timepoints)
        for j in range(num_channels):
            for k in range(num_timepoints):
                if j % 2 == 0:
                    self.positional_encoding[j][k] =\
                        torch.sin(torch.tensor(k) / (10000 ** (torch.tensor(j) / num_channels)))
                else:
                    self.positional_encoding[j][k] =\
                        torch.cos(torch.tensor(k) / (10000 ** ((torch.tensor(j) - 1) / num_channels)))

        # Multi-Head Self Attention
        self.multihead_attn = nn.MultiheadAttention(embed_dim=num_channels,
                                                    num_heads=num_heads)

        # Feed-Forward Network
        self.ffn = nn.Sequential(
            nn.Linear(num_channels, intermediate_dim),
            nn.ReLU(),
            nn.Linear(intermediate_dim, ffn_output_dim)
        )

        # Layer Normalization
        self.norm1 = nn.LayerNorm(num_channels)
        self.norm2 = nn.LayerNorm(num_channels)

        # Classifier
        self.classifier = nn.Linear(num_channels * num_timepoints, output_dim)

    def forward(self, X):
        # Input Standardization
        mean = X.mean(dim=2, keepdim=True)
        std  = X.std(dim=2, keepdim=True)
        X_hat = (X - mean) / (std + 1e-5)  # epsilon to avoid division by zero

        # Add Positional Encoding
        X_tilde = X_hat + self.positional_encoding.to(X.device)

        # Reshape for multi-head self attention: (seq_len, batch_size, embed_dim)
        X_tilde = X_tilde.permute(2, 0, 1)

        # Multi-Head Self Attention
        attn_output, _ = self.multihead_attn(X_tilde, X_tilde, X_tilde)

        # Reshape back and Apply Layer Norm
        # attn_output = attn_output.permute(1, 2, 0)  # Reshape: (batch_size, embed_dim, seq_len)
        X_ring = torch.stack([self.norm1(a) for a in attn_output], dim=1)

        # Position-wise Feed-Forward Networks
        ff_output = self.ffn(X_ring)
        O = self.norm2(ff_output + X_ring)

        # Classifier
        # Flatten and classify
        O_flat = O.view(O.size(0), -1)  # Flatten the tensor
        output = self.classifier(O_flat)

        return output

In [ ]:
# Sample generated data (replace with real EEG (segmented) data)
num_channels=32
num_timepoints=200
batch_size = 64

X = torch.randn(batch_size, num_channels, num_timepoints)
y = torch.randint(0, 2, (batch_size,))  # L=2 for binary classification

# Model, Loss and Optimizer
model = EEGTransformer(num_channels, num_timepoints, output_dim=2,
                       hidden_dim=512, num_heads=8, key_query_dim=512,
                       hidden_ffn_dim=512, intermediate_dim=2048,
                       ffn_output_dim=num_channels)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


# once the model is trained, it can be tested on unseen EEG test examples
# also, different model selection techniques (e.g. cross-validation methods) can be implemented within the training loop


### Transformer's Architecture for EEG Classification

This section presents the "standard" approach for utilizing the Transformer encoder to classify EEG patterns for BCIs.

#### Input Standardization and Positional Encoding

Let the set of pairs $ D_{\text{train}} = \{(\mathbf{X}_1,{y}_1),\dots, (\mathbf{X}_n,{y}_n)\} $ denote $ n $ trials of EEG recordings where $ {y}_i $ is the scaler class variable with $ L $ possible labels (e.g., target and non-target in a binary classification) and $ \mathbf{X}_i\in \mathbb{R}^{c\times p} $ is the collection of EEG observations in the $ i^{\text{th}} $ trial over $ c $ channels and $ p $ time points; that is to say,

$$ \mathbf{X}_i=[\mathbf{x}_{i1},\mathbf{x}_{i2},\ldots,\mathbf{x}_{ic}]^T, i=1,\ldots,n\,, $$
 
with $ \mathbf{x}_{ij}=[x_{ij1}, \ldots, x_{ijp}]^T \in \mathbb{R}^{p\times 1}, j=1,\ldots,c $, where $ x_{ijk}, k=1, \ldots, p $ denotes the $ k^{\text{th}} $ element of vector $ \mathbf{x}_{ij} $, and $ T $ denotes the transpose operator. The goal is to use $ D_{\text{train}} $ and train a classifier $ \psi: \mathbb{R}^{c\times p} \rightarrow \{0, 1, \ldots, L-1\} $ that maps a given $ \mathbf{X} $ to a possible value of the class variable.

It is common to apply standardization for each channel to make the sensory data across all channels comparable. 
In this regard, each $ \mathbf{X}_i $ is converted to $ \hat{\mathbf{X}}_i $ where 

$$ \hat{\mathbf{X}}_i=[\hat{\mathbf{x}}_{i1},\hat{\mathbf{x}}_{i2},\ldots,\hat{\mathbf{x}}_{ic}]^T, i=1,\ldots,n\,, $$

and where $ \hat{\mathbf{x}}_{ij} = [\hat{x}_{ij1}, \ldots, \hat{x}_{ijp}]^T $ such that 

$$ \hat{x}_{ijk} = \frac{{x}_{ijk}-m_{ij}}{s_{ij}}\,, $$

with $ m_{ij} $ and $ s_{ij} $ being the sample mean and sample standard deviation of vector $\mathbf{x}_{ij} $ given by

$$ m_{ij} = \frac{1}{p} \sum_{k=1}^p {x}_{ijk}\,, $$
$$ s_{ij} = \sqrt{\frac{1}{p} \sum_{k=1}^p ({x}_{ijk}-m_{ij})^2}\,, $$

respectively.

In order for the Transformer to make use of EEG recording orders, it is common to encode some information about the position of sequence elements in its input \cite{vaswani_attention_2017}. This positional encoding is generally realized by adding each $ \hat{\mathbf{X}}_i $ to a matrix $ \mathbf{P} \in \mathbb{R}^{c\times p} $ that is defined based on trigonometric functions with different frequencies for each channel \cite{vaswani_attention_2017}. As a result, we obtain

$$ \tilde{\mathbf{X}}_i = \hat{\mathbf{X}}_i + \mathbf{P}, \,i=1,\ldots, n, $$

where the element on row (channel) $ j=1,\ldots, c $, and column (time index) $ k=1, \ldots, p $, of $ \mathbf{P} $, denoted $ p_{jk} $ is given by

$$ p_{jk} = \begin{cases}
\text{

sin}\Big(k/10000^{j/c} \Big), & \text{for even } j \\
\text{cos}\Big(k/10000^{j-1/c} \Big), & \text{for odd } j
\end{cases} $$



#### Self-Attention Mechanisms: Capturing Contexts for EEG Classification

Capturing contexts is the essential concept that makes attention mechanism a promising operation for EEG classification. A context is simply another representation of an element of the input sequence (here one column of each $ \tilde{\mathbf{X}}_i $) based on its compatibility with other elements within the sequence. The most widely used attention operation for EEG classification is scaled dot-product self-attention, denoted $ \text{SA}^d_{\mathbf{V}, \mathbf{K}, \mathbf{Q}}(\tilde{\mathbf{X}}_i): \mathbb{R}^{c\times p} \rightarrow \mathbb{R}^{d\times p} $, which was initially proposed and used for translation tasks \cite{vaswani_attention_2017}. In particular,

$$ \text{SA}_{\mathbf{V}, \mathbf{K}, \mathbf{Q}}^d(\tilde{\mathbf{X}}_i) = \mathbf{V}\tilde{\mathbf{X}}_i\times\text{softmax}\Big(\frac{\tilde{\mathbf{X}}_i^T\mathbf{K}^T\mathbf{Q}\tilde{\mathbf{X}}_i}{\sqrt{q}}\Big)\,, $$

where $ \mathbf{V} \in \mathbb{R}^{d\times c} $, $ \mathbf{K} \in \mathbb{R}^{q\times c} $, $ \mathbf{Q} \in \mathbb{R}^{q\times c} $ are projection matrices that are learned in the training process, $ q $ is known as attention dimensionality, and  $ d $, which is generally a tuning parameter, denotes the dimensionality of the columns of the output matrix (context vectors). We use superscript $ d $ in $ \text{SA}_{\mathbf{V}, \mathbf{K}, \mathbf{Q}}^d(\tilde{\mathbf{X}}_i) $ to highlight the dimensionality of context vectors.



#### Multi-Head Self-Attention

Rather than a single self-attention operation, it is generally beneficial to apply multiple self-attentions in parallel. Using this operation, we view the compatibility of sequence elements using different learned projections. In this context, it is also common to refer to the output matrix of each self-attention as a head. In particular, the multi-head self-attention, denoted $ \text{MSHA}(\tilde{\mathbf{X}}_i) : \mathbb{R}^{c\times p} \rightarrow \mathbb{R}^{d_h\times p} $, is defined as

$$ \text{MSHA}^{d_h}(\tilde{\mathbf{X}}_i) = \mathbf{W}[\text{SA}^d_{\mathbf{V}_1, \mathbf{K}_1, \mathbf{Q}_1}(\tilde{\mathbf{X}}_i)^T, \ldots, \text{SA}^d_{\mathbf{V}_m, \mathbf{K}_m, \mathbf{Q}_m}(\tilde{\mathbf{X}}_i)^T]^T\,, $$

where $ \mathbf{W}\in \mathbb{R}^{d_h\times md} $ is another learnable projection matrix, $ m $ is the number of self-attentions used in (\ref{MHSA}), which is also known as the number of heads, and $ d_h $ is the dimensionality of columns in the output of $ \text{MSHA}^{d_h}(\tilde{\mathbf{X}}_i) $ operation.



#### Identity Skip-Connection and Layer Normalization

To ensure the stability and efficacy of the training process, especially with the complex nature of EEG data, the Transformer encoder utilizes identity skip-connections \cite{he_deep_2015} followed by layer normalization \cite{Jimmy_2016}. Here we define these operations. Let $ \text{SKP}\big({\text{LAY}(\mathbf{Y})}\big): \mathbb{R}^{a\times b}\rightarrow \mathbb{R}^{a\times b}$ denote the identity skip-connection around a layer $\text{LAY}(\mathbf{Y}) $ (an operation) that operates on an input $ \mathbf{Y} \in \mathbb{R}^{a\times b} $ to produce an output of the same size as the input. Then

$$ \text{SKP}\big({\text{LAY}(\mathbf{Y})}\big) = \mathbf{Y} + \text{LAY}(\mathbf{Y})\,. $$

That is to say, we simply add the output of $ \text{LAY}(\mathbf{Y}) $ to its input. 
Furthermore, let $\text{LN}(\mathbf{Y}):\mathbb{R}^{a\times b} \rightarrow \mathbb{R}^{a\times b}$ denote the layer normalization applied to an $ (a > 1)\times b $ matrix $ \mathbf{Y} $ with elements $ y_{jk}, j=1,\ldots,a, k=1,\ldots,b $ where each row records measurements for a "features" (here, channel). Then, $ \text{LN}(\mathbf{Y}) $ produces $ \mathring{\mathbf{Y}} $, which is a matrix of the same size $ \mathring{\mathbf{Y}} $ with elements $ \mathring{y}_{jk} $ where

$$ \mathring{y}_{jk} = \frac{{y}_{jk}-m_{k}}{s_{k}}\,, $$

and where

$$ m_{k} = \frac{1}{a} \sum_{j=1}^a {y}_{jk}\,, $$
$$ s_{k} = \sqrt{\frac{1}{a} \sum_{j=1}^a ({y}_{jk}-m_{k})^2}\,. $$

In other words, $ \mathring{\mathbf{Y}} $ is a type of standardization where the sample mean and sample standard deviation are computed for each column of $ \mathbf{Y} $ (in the EEG context means for each time point in the sequence) over all features. One place that these operations are used in the transformer encoder is to produce $ \mathring{\mathbf{X}}_i $ as follows:

$$ \mathring{\mathbf{X}}_i = \text{LN}\Big(\text{SKP}\big({\text{MSHA}^{c}(\tilde{\mathbf{X}}_i)}\big)\Big)\,; $$
 

that is, the skip-connection is used around the multi-head self-attention, which is then followed by layer normalization. Note that the use of skip-connection in (\ref{outSKPLN}) enforces setting $ d_h $ defined in (\ref{MHSA}) to $ c $, which is the number of channels.



#### Position-wise Feed-Forward Networks

The Transformer encoder utilizes a fully connected feed-forward network that transforms each element of a given sequence individually. Let $ \mathbf{Y} \in \mathbb{R}^{a\times b} $ be the generic matrix defined before. The effect of this position-wise feed-forward network operated on an input $ \mathbf{Y} $, denoted 
$\text{FFN}(\mathbf{Y})$, is:

$$ \text{FFN}^s(\mathbf{Y}) = [g(\mathbf{y}_1), \ldots, g(\mathbf{y}_b)]\,, $$
 

where $ \mathbf{y}_k, k=1,\ldots, b $ are columns of $ \mathbf{Y} $ and

$$ g(\mathbf{y}_k) = \mathbf{W}_2\times f(\mathbf{W}_1\mathbf{y}_k + \mathbf{b}_1) + \mathbf{b}_2\,, $$

where $ f(.) $ denotes an element-wise nonlinear activation function (e.g., ReLU), and $\mathbf{W}_1\in \mathbb{R}^{r\times a}$, and $\mathbf{W}_2 \in \mathbb{R}^{s\times r}$, and $\mathbf{b}_1 \in \mathbb{R}^{r\times 1}$, and $\mathbf{b}_1 \in \mathbb{R}^{s\times 1} $ are learnable matrices and vectors—$ r $ is generally a tuning parameter. 

We use superscript $ s $ in $ \text{FFN}^s(\mathbf{Y}) $ to highlight the dimensionality of output vectors in (\ref{FFN}). In the Transformer encoder, position-wise feed-forward network is used to produce an output $ {\mathbf{O}}_i $ from $ \mathring{\mathbf{X}}_i $ obtained in (\ref{outSKPLN}), which is then added to its input through the skip-connection, followed by layer normalization. This operation is characterized as follows:

$$ {\mathbf{O}}_i = \text{LN}\Big(\text{SKP}\big({\text{FFN}^{c}(\mathring{\mathbf{X}}_i)}\big)\Big)\,. $$
 

Note that the use of skip-connection in (\ref{outSKPLNN}) enforces setting $ s $ defined in (\ref{FFN}) to $ c $. The classification can be performed by vectorizing $ {\mathbf{O}}_i $ and using that as the input to a fully connected layer with a softmax activation function. 

In [1]:
def replace_latex_delimiters(text):
    # Replace \( and \) with $
    text = text.replace(r"\(", "$").replace(r"\)", "$")
    
    # Replace \[ and \] with $$
    text = text.replace(r"\[", "$$").replace(r"\]", "$$")
    
    return text


# Read input text
with open('eq.txt', 'r') as file:
    content = file.read()

# Apply the replacement
modified_content = replace_latex_delimiters(content)

 
with open('output.txt', 'w') as file:
    file.write(modified_content)
